In [21]:
class HandlerRegistry:
    def __init__(self):
        self._map = {}

    def register(self, event_type, event_message_class):
        if event_type in self._map:
            raise KeyError(f"event_type '{event_type}' 已經被註冊")

        def decorator(cls):
            self._map[event_type] = (cls(), event_message_class)
            return cls
        return decorator

    def get_handler(self, event_type):
        entry = self._map.get(event_type)
        return entry[0] if entry else None

    def get_event_message_class(self, event_type):
        entry = self._map.get(event_type)
        return entry[1] if entry else None

# 1. 建立註冊中心實例
handler_registry = HandlerRegistry()

In [3]:

from abc import ABC, abstractmethod


class EventHandlerBase(ABC):

    def __init__(self, message_queue_client: str = "default"):
        """
        Base class for event handlers.
        :param message_queue_client: An instance of MessageQueueClientBase to send messages.
        """
        self.message_queue = message_queue_client

    @abstractmethod
    async def handle(self, event_object, event):
        pass


In [12]:
class UserCreatedMessage:
    def __init__(self, user_id, user_name, user_age):
        self.user_id = user_id
        self.user_name = user_name
        self.user_age = user_age

In [18]:
from enum import Enum, auto


class Type(Enum):
    USER_CREATED = auto()

In [22]:
@handler_registry.register(event_type=Type.USER_CREATED.name, event_message_class=UserCreatedMessage)
class UserCreationHandler(EventHandlerBase):
    async def handle(self, event_object: UserCreatedMessage, event):
        print(f"處理中... 新使用者已建立！ID: {event_object.user_id}, 名稱: {event_object.user_name}")

In [26]:
event_type_from_message_queue = "USER_CREATED"
event_data = {"user_id": 123, "user_name": "Alice", "user_age": 14}

In [27]:
handler = handler_registry.get_handler(event_type_from_message_queue)
message_class = handler_registry.get_event_message_class(event_type_from_message_queue)

# 5. 執行處理
if handler and message_class:
    message_instance = message_class(**event_data)
    await handler.handle(message_instance, message_instance)
else:
    print(f"找不到事件 '{event_type_from_message_queue}' 的處理器")

處理中... 新使用者已建立！ID: 123, 名稱: Alice
